In [1]:
import alpaca_trade_api as tradeapi
import time
import requests
import logging

In [2]:
def send(text):
    token = '<insert your access token>'
    your_id = '<insert your id>'
    params = {'chat_id': your_id, 'text': text, 'parse_mode': 'HTML'}
    resp = requests.post('https://api.telegram.org/bot{}/sendMessage'.format(token), params)
    resp.raise_for_status()

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s: %(levelname)s: %(message)s')

In [4]:

api = tradeapi.REST('<insert api ID>', '<insert secret>', base_url='https://paper-api.alpaca.markets') 
account = api.get_account()
account

# https://github.com/alpacahq/alpaca-trade-api-python/tree/cd22b3393aff8df214d867b6a4723a21ea34a3c0

Account({   'account_blocked': False,
    'account_number': 'PA2E0F7YERPO',
    'buying_power': '456753.56',
    'cash': '113665.85',
    'created_at': '2020-11-17T17:28:00.960389Z',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '456753.56',
    'equity': '114363.37',
    'id': '7f81289d-15ed-40ed-85a3-1e831cce3b12',
    'initial_margin': '348.76',
    'last_equity': '114362.57',
    'last_maintenance_margin': '0',
    'long_market_value': '697.52',
    'maintenance_margin': '209.256',
    'multiplier': '4',
    'pattern_day_trader': False,
    'portfolio_value': '114363.37',
    'regt_buying_power': '228029.22',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [5]:
order_sent = None

while not order_sent:

    barset = api.get_barset('TSLA',
               'day',
               limit = 10).df['TSLA']

    today_close = barset['close'][-1]
    five_days_ago_close = barset['close'][-6]

    percent_change = (today_close - five_days_ago_close)/five_days_ago_close*100

    order_info = None
    direction = 'buy'
    
    if percent_change > 5:
        order_info = api.submit_order(
        symbol='TSLA',
        side=direction,
        type='market',
        qty='1',
        time_in_force='day',
        )
        time.sleep(3)
        if order_info.status == 'accepted':
            order_sent = True
            logging.info(f'Order status for {direction} trade: {order_info.status}') # Feel free to include details such as order entry price.
            send(f'Order status for {direction} trade: {order_info.status}') # Feel free to include details such as order entry price.
    
    print(f'TSLA current price: {today_close}')
    print(f'Percent change for TSLA over the last 5 days: {percent_change}')
    time.sleep(10)
    
order_info

2021-04-28 23:28:11,877: INFO: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-04-28 23:28:11,879: INFO: NumExpr defaulting to 8 threads.
2021-04-28 23:28:15,273: INFO: Order status for buy trade: accepted


TSLA current price: 697.21
Percent change for TSLA over the last 5 days: -6.277641112500159


KeyboardInterrupt: 